In [1]:
import re
import glob
import json
import utils
import spacy.lang.en as en

from tqdm import tqdm
from collections import defaultdict

In [2]:
nlp = en.English()
tokenizer = nlp.tokenizer

In [3]:
# postags = utils.read_file("/home/km55359/rawdata/books1/postags.txt")
postags = utils.read_file("/home/km55359/rawdata/babylm_data/babylm_100M/sents/postags.txt")
sents = utils.read_file("/home/km55359/rawdata/babylm_data/babylm_100M/sents/babylm_sents.txt")
# sents = []
# train_files = glob.glob('/home/km55359/rawdata/books1/epubtxt/*.txt')
# for file in train_files:
#     sents.extend(utils.read_file(file))

In [4]:
key_regex = re.compile(r"\b(JJ|JJR) IN (PRP|PRP\$) (VBP|VBZ|MD VB) (,|\.)")

In [5]:
# found = []
# found_pos = []
# for i, entry in enumerate(tqdm(postags)):
#     if key_regex.search(entry):
#         # print(entry)
#         # print(sents[postags.index(entry)])
#         found_pos.append(entry)
#         found.append(sents[i])

found_openbooks = []

for i, batch in enumerate(tqdm(zip(sents, postags))):
    sent, postag = batch
    sent_split, postag_split = [t.text for t in tokenizer(sent)], postag.split()
    
    keys = key_regex.search(postag)
    if keys:
        idx = len(postag[:keys.start()].split())
        prep = sent_split[idx+1]
        adj = sent_split[idx]
        pron = sent_split[idx+2]
        verb = sent_split[idx+3]
        if prep.lower() in ["than", "as"]:
            # found_openbooks.append({
            #     'sentence_idx': i,
            #     'PREP': prep,
            #     'ADJ': adj,
            #     'PRON': pron,
            #     'VERB': verb,
            #     'sentence': sent
            # })
            found_openbooks.append([i, prep, adj, pron, verb, sent])
        # if det.lower() in ["a", "an", "another"]:
        #     keys.append({
        #         'sentence_idx': i,
        #         'DT': det,
        #         'ADJ': adj,
        #         'sentence': sent
        #     })

0it [00:00, ?it/s]

11632617it [03:04, 63076.01it/s]


In [6]:
len(found_openbooks)
# BabyLM: 2913
# openbooks: 34831

2913

In [7]:
import csv
with open('../data/babylm-analysis/keys_to_pipps.csv', 'w') as csvfile:
    fieldnames = ['sentence_idx', 'PREP', 'ADJ', 'PRON', 'VERB', 'sentence']
    writer = csv.writer(csvfile)
    writer.writerow(fieldnames)
    for row in found_openbooks:
        writer.writerow(row)

In [10]:
# write to csv
# import csv
# with open('../data/pipps/openbooks_keys_to_pipps.csv', 'w') as csvfile:
#     fieldnames = ['sentence_idx', 'PREP', 'ADJ', 'PRON', 'VERB', 'sentence']
#     writer = csv.writer(csvfile)
#     writer.writerow(fieldnames)
#     for row in found_openbooks:
#         writer.writerow(row)

In [16]:
# write found_openbooks to jsonl without using utils
# utils.write_jsonl(found_openbooks, "../data/pipps/openbooks_keys_to_pipps.jsonl")
with open("../data/pipps/openbooks_keys_to_pipps.jsonl", "w") as f:
    for entry in found_openbooks:
        f.write(json.dumps(entry) + "\n")

In [15]:
found[10:15]

[{'sentence_idx': 47844,
  'PREP': 'as',
  'ADJ': 'good',
  'PRON': 'it',
  'VERB': 'could',
  'sentence': 'Now they had everything that they wanted, and everything was as good as it could be.'},
 {'sentence_idx': 47984,
  'PREP': 'than',
  'ADJ': 'stronger',
  'PRON': 'I',
  'VERB': 'am',
  'sentence': '“Well,” said the East Wind, “I have heard tell of the Prince, and of his castle, but I do not know the way to it, for I have never blown so far; but, if you like, I will go with you to my brother the West Wind: he may know that, for he is much stronger than I am.'},
 {'sentence_idx': 48270,
  'PREP': 'as',
  'ADJ': 'much',
  'PRON': 'they',
  'VERB': 'can',
  'sentence': 'Yeah and, and concerned that, in as much as they can be.'},
 {'sentence_idx': 48725,
  'PREP': 'as',
  'ADJ': 'clever',
  'PRON': 'she',
  'VERB': 'is',
  'sentence': '“And what do you want with her, pretty one?” said the little monster, “for I am a friend of hers, and, for the matter of that, I am quite as clever as 

In [10]:
for f in found:
    print(f['sentence'])
    print("")

Well that's it for our first look at our new building, I hope you're as excited as I am, and I hope to come back in a month or so and give you an updated look at our new building.

Iím cleverer than I look.

No one in the whole world is so sensitive as I am, I am quite sure of that.”

I , unworthy as I am , represent the sole hope of the royal family .

“There's many a one considerably older than I am.”

Their brother, who is older than they are, stands up in the swing; he twines his arms round the cords to hold himself fast, for in one hand he has a little cup, and in the other a clay-pipe.

I , unworthy as I am , represent the sole hope of the royal family .

“Well,” said the East Wind, “I have heard tell of the Prince, and of his castle, but I do not know the way to it, for I have never blown so far; but, if you like, I will go with you to my brother the West Wind: he may know that, for he is much stronger than I am.

Just try to take it off, and you will soon find out that I am mo

In [9]:
aadj_regex = re.compile(r"\bDT (JJ|JJR|JJS)")
anum_regex = re.compile(r"\bDT CD")
adjnum_regex = re.compile(r"\b(JJ|JJR|JJS) CD")
numadj_regex = re.compile(r"\bCD (JJ|JJR|JJS)")

In [15]:
aadj = 0
anum = 0
adjnum = 0
numadj = 0

for entry in tqdm(postags2):
    if aadj_regex.search(entry):
        aadj += 1
    if anum_regex.search(entry):
        anum += 1
    if adjnum_regex.search(entry):
        adjnum += 1
    if numadj_regex.search(entry):
        numadj += 1

100%|██████████| 11632617/11632617 [00:18<00:00, 627347.58it/s]


In [16]:
numadj/adjnum

1.6207073894850037